In [7]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['INTUX']
collection = db['GazeData']

# Aggregation pipeline to normalize timestamps and update the collection
pipeline = [
    {
        '$group': {
            '_id': '$rollNo',
            'minTimestamp': {'$min': '$timestamp'},
            'data': {'$push': '$$ROOT'}
        }
    },
    {
        '$project': {
            '_id': 0,
            'rollNo': '$_id',
            'data': {
                '$map': {
                    'input': '$data',
                    'as': 'doc',
                    'in': {
                        '$mergeObjects': [
                            '$$doc',
                            {
                                'normalizedTimestamp': {
                                    '$subtract': ['$$doc.timestamp', '$minTimestamp']
                                }
                            }
                        ]
                    }
                }
            }
        }
    },
    {
        '$unwind': '$data'
    },
    {
        '$replaceRoot': {'newRoot': '$data'}
    },
    {
        '$project': {'minTimestamp': 0}
    },
    {
        '$out': 'GazeData'  # Write the results back to the same collection
    }
]

# Execute the aggregation pipeline
collection.aggregate(pipeline)

# Print a message indicating that the update is complete
print("Normalization and update completed.")


Normalization and update completed.


In [8]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['INTUX']
collection = db['ClickData']

# Aggregation pipeline to normalize timestamps and update the collection
pipeline = [
    {
        '$group': {
            '_id': '$rollNo',
            'minTimestamp': {'$min': '$timestamp'},
            'data': {'$push': '$$ROOT'}
        }
    },
    {
        '$project': {
            '_id': 0,
            'rollNo': '$_id',
            'data': {
                '$map': {
                    'input': '$data',
                    'as': 'doc',
                    'in': {
                        '$mergeObjects': [
                            '$$doc',
                            {
                                'normalizedTimestamp': {
                                    '$subtract': ['$$doc.timestamp', '$minTimestamp']
                                }
                            }
                        ]
                    }
                }
            }
        }
    },
    {
        '$unwind': '$data'
    },
    {
        '$replaceRoot': {'newRoot': '$data'}
    },
    {
        '$project': {'minTimestamp': 0}
    },
    {
        '$out': 'ClickData'  # Write the results back to the same collection
    }
]

# Execute the aggregation pipeline
collection.aggregate(pipeline)

# Print a message indicating that the update is complete
print("Normalization and update completed.")


Normalization and update completed.


In [7]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['INTUX']
gaze_data_collection = db['GazeData']
meta_data_collection = db['MetaData']

# Get unique users from MetaData collection
users = meta_data_collection.distinct('rollNo')

# Iterate over each user
for user in users:
    # Find the user's metadata
    metadata = meta_data_collection.find_one({'rollNo': user})
    if metadata:
        # Get screen dimensions
        screen_width = metadata['screenWidth']
        screen_height = metadata['screenHeight']

        # Update GazeData collection for the user
        gaze_data_collection.update_many(
            {'rollNo': user},
            [
                {
                    '$set': {
                        'normalizedX': {'$divide': ['$x', screen_width]},
                        'normalizedY': {'$divide': ['$y', screen_height]}
                    }
                }
            ]
        )

print("Normalization of x and y coordinates completed.")


Normalization of x and y coordinates completed.


In [8]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['INTUX']
gaze_data_collection = db['ClickData']
meta_data_collection = db['MetaData']

# Get unique users from MetaData collection
users = meta_data_collection.distinct('rollNo')

# Iterate over each user
for user in users:
    # Find the user's metadata
    metadata = meta_data_collection.find_one({'rollNo': user})
    if metadata:
        # Get screen dimensions
        screen_width = metadata['screenWidth']
        screen_height = metadata['screenHeight']

        # Update GazeData collection for the user
        gaze_data_collection.update_many(
            {'rollNo': user},
            [
                {
                    '$set': {
                        'normalizedX': {'$divide': ['$x', screen_width]},
                        'normalizedY': {'$divide': ['$y', screen_height]}
                    }
                }
            ]
        )

print("Normalization of x and y coordinates completed.")


Normalization of x and y coordinates completed.
